In [88]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm_notebook
import re
import os

In [44]:
BASE_DIR = os.path.realpath(os.getcwd()+'../../..')

In [4]:
not_include = ['cover', 'remix', 'live'] # Запрещённые слова для названий песен todo: 'Live'?
max_eps = 1 # Максимальное отклонение от заданной длины дорожки (в секундах)

# Функции для парсинга

Преобразование секунд в минуты и обратно

In [7]:
def secs_to_string(secs):
    m = str(secs//60)
    m = '0'+m if len(m)==1 else m
    s = str(secs%60)
    s = '0'+s if len(s)==1 else s
    return ':'.join([m,s])

In [12]:
def string_to_secs(string):
    m,s = map(int,string.split(':'))
    return m*60+s

Нахождение информации о длительности музыки (?на отдельном сайте?)

In [17]:
def parse_duration(track_html):
    #print 'parsing'
    d = track_html.find('em')
    #print 'asdf'
    while True:
        try:
            d.div.decompose()
        except:
            break
    #print d
    while True:
        try:
            d.a.decompose()
        except:
            break
    #print d
    return d.text.split()[0]

Преобразование имени файла для адресной строки (наверняка это лишний велосипед)

In [144]:
def format_fname(s, space_replacer='_'):
    s = re.sub(r'\(|\)|\'', '', s)
    #for c in ['(',')','\'']:
    #    s = s.replace(c,'')
    s = s.strip(' ')
    s = re.sub(r' ', space_replacer, s)
    return s

Загрузка записи в определённый файл

In [145]:
def download_file(artist, title, number, href):
    #dname = 'audio/billboard/'+'0'*(4-len(str(number)))+str(number)+'/'
    dname = '/audio/'
    fname = format_fname('{}_{}-{}.mp3'.format(number, artist,title))
    fname = re.sub(r'&', '_and_', fname)
    try:
        !wget -q -nc -t 1 -O {BASE_DIR+dname+fname} '{href}'
    except:
        raise Exception('Found but not downloaded')
        #ds.at[i,'status']='Found but not downloaded'
        #print('Found but not downloaded')
        #continue

Парсинг сайта ipleer.fm

In [71]:
def parse_ipleer(artist, title, duration):
    SEARCH_URL = 'https://ipleer.fm/search/q/'
    SONGS_URL = 'https://ipleer.fm'

    url = format_fname('{}{}+{}/'.format(SEARCH_URL, artist, title), space_replacer='+')
    list_page = requests.get(url=url)
    
    if list_page.status_code != 200:
        raise Exception('List_page_not_found')
    
    list_soup = BeautifulSoup(list_page.text, 'html.parser')
    tracks = list_soup.find_all('li',class_='track')
    ntracks = []
    for track_id,track in enumerate(tracks):
        try:
            a = track.find('span',class_='cartist').text
            t = track.find('span',class_='ctitle').text
            d = parse_duration(track)
            h = track.find('a',class_='playlist-down').get('href')
        except:
            a,t,d, h = None, None, None, None
        if a and t and d and h:
            # Если кавер или ремикс (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in t.lower(), not_include)))>0\
             or len(list(filter(lambda x: x in a.lower(), not_include)))>0:
                continue
            # Если автор не совпадает с заданным (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in a, artist.split()))) != len(artist.split())\
             and len(list(filter(lambda x: x in t, artist.split()))) != len(artist.split()):
                continue
            # Если название не совпадает с заданным (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in t, title.split()))) != len(title.split())\
             and len(list(filter(lambda x: x in a, title.split()))) != len(title.split()):
                continue
            try:
                d = string_to_secs(d)-string_to_secs(duration)
            except:
                continue
            if abs(d) > max_eps:
                continue
           
            ntracks.append({'artist':a, 'title':t, 'time_delta':d, 'href':h, 'track':track})
    
    if len(ntracks)==0:
        raise Exception('No_correct_songs_found',url)
    
    ntracks = sorted(ntracks, key= lambda x: x['time_delta'])
    
    dur_eps = 0
    for track in ntracks:
        download_page = requests.get(SONGS_URL+track['href'])
        if download_page.status_code == 200:
            dur_eps = track['time_delta']
            break
    if download_page.status_code != 200:
        raise Exception('Download_page_not_found')
        
    download_soup = BeautifulSoup(download_page.text, 'html.parser')
    download_href = download_soup.find('a',class_='onesongblock-download').get('href')
    
    return download_href, download_page.url, dur_eps

In [72]:
parse_ipleer('Chicago','25 or 6 to 4','4:50')
#print max_eps

('http://data.iplayer.info/file/11040834/ejRaU1hoczJaRGtBT2FjTGpyb0EzaTZhRk1vN3hxQmV2cHJQNDhrck5wM0F4L0hRT21jUjd6QnBkYnBCR3daVEFpUURIZnhRaFR3SlYvT1E5UlR2d3ViN2lMSWV6TzZFWEExWkVGQmJUbGw4azl0aU9yM09uWFczRUJCcThSUzg/Chicago_-_25_or_6_to_4_(iPlayer.fm).mp3',
 'https://ru123.iplayer.info/song/11040834/Chicago_-_25_or_6_to_4/',
 1)

# Парсинг сайта Ipleer.fm

Загрузка БД песен

In [85]:
songs_ds_fname = BASE_DIR+'/dsets/songs_ds.csv'

In [107]:
ds = pd.read_csv(songs_ds_fname, index_col=0)
ds['t_eps']=0.0
ds['href']='nan'
ds['status']='nan'
ds.head()

,title,artist,number,duration,t_eps,href,status
0,Chicago,25 or 6 to 4,176,4:50,0.0,nan,nan
1,Sukiyaki,A Taste Of Honey,1286,3:43,0.0,nan,nan
2,Chiquitita,ABBA,183,5:25,0.0,nan,nan
3,"Knowing Me, Knowing You",ABBA,231,4:04,0.0,nan,nan
4,Honey Honey,ABBA,902,2:54,0.0,nan,nan


Количество незагруженных песен

In [117]:
ds[ds['status']!='Downloaded'].shape[0]

530

Вынесение всех незагруженных песен в отдельный датасет

In [158]:
not_load = ds[ds['status']!='Downloaded'].append(ds[ds['t_eps']>max_eps])

In [159]:
%%time
#not_load = ds[ds['status']!='Downloaded']

for row in tqdm_notebook(not_load.iterrows(), total=len(not_load)):
    i,row = row
    #print(i,)
    # Обновляем файл датасета
    if i%5==0:
        ds.to_csv(songs_ds_fname)
    
    try:
        download_href, page_url, dur_eps = parse_ipleer(artist=row.artist, title=row.title, duration=row.duration)
        download_file(artist=row.artist, title=row.title, number=row.number, href=download_href)
        
        ds.at[i,'status']='Downloaded'
        ds.at[i,'href']=page_url
        ds.at[i,'t_eps']=dur_eps
        #print('+')
    except Exception as e:
        ds.at[i,'status']=e.args[0]
        if len(e.args)==2:
            ds.at[i,'href']=e.args[1]
        #print(e.args[0])

A Jupyter Widget


CPU times: user 10min 27s, sys: 1min 54s, total: 12min 21s
Wall time: 3h 2min 58s


In [160]:
ds.to_csv(songs_ds_fname)

In [169]:
not_load = ds[ds['status']!='Downloaded'].append(ds[ds['t_eps']>max_eps])

In [170]:
not_load

,title,artist,number,duration,t_eps,href,status
6,On And On And On,Abba,894,3:42,0.0,https://ipleer.fm/search/q/Abba+On+And+On+And+On/,No_correct_songs_found
7,Take A Chance On Me,Abba,961,4:02,0.0,https://ipleer.fm/search/q/Abba+Take+A+Chance+...,No_correct_songs_found
11,Undercover Angel,Alan O'Day,367,3:33,0.0,https://ipleer.fm/search/q/Alan+ODay+Undercove...,No_correct_songs_found
12,Schoool's out,Alice Cooper,195,4:21,0.0,https://ipleer.fm/search/q/Alice+Cooper+Schooo...,No_correct_songs_found
14,Naked Eyes,Always Something There to Remind Me,181,3:41,0.0,https://ipleer.fm/search/q/Always+Something+Th...,No_correct_songs_found
16,Giving You the Best That I Got,Anita Baker,578,4:21,0.0,https://ipleer.fm/search/q/Anita+Baker+Giving+...,No_correct_songs_found
19,I Can't Stand The Rain,Ann Peebles,377,2:34,0.0,nan,List_page_not_found
20,Could I Have This Dance,Anne Murray,199,5:03,0.0,https://ipleer.fm/search/q/Anne+Murray+Could+I...,No_correct_songs_found
22,A Love Song,Anne Murray,917,2:44,0.0,https://ipleer.fm/search/q/Anne+Murray+A+Love+...,No_correct_songs_found
27,How Blue Can You Get,B.B. King,510,2:43,0.0,https://ipleer.fm/search/q/B.B.+King+How+Blue+...,No_correct_songs_found
